In [104]:
%matplotlib widget

In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
from scipy import stats

In [106]:
font = {'family' : 'serif',
        'serif' : 'Times New Roman',
        'size'   : 8}
plt.rc('font', **font)

In [107]:
server = 'mrwhite.intrans.iastate.edu'
database = 'sim_Andalib'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes')

In [108]:
### Simulation Information

simulation_start = 300
simulation_end = 3600

In [109]:
## check the filtering conditions. May need to change these everytime a new model is being implemented.

sim_id = 21
run_id = 0
iteration = 10
number_of_intersection = 10
cycle_length = 120

number_of_cycle = simulation_end//cycle_length
cycle_remainder = simulation_end%cycle_length

In [110]:
directory = "C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\analysis\\"
tt_definitions = pd.read_excel(directory + "vissim_definitions.xlsx", sheet_name = "38th_street", dtype="Int64")
tt_definitions = tt_definitions.fillna(0)

In [111]:
# run_ids = [13, 2, 14, 5,7,23]
delay_sql = "SELECT [RunID],[Iteration],[Time],[No_],[Veh],[Trav_],[Delay_] FROM [sim_Andalib].[dbo].[traveltimes] where [StudyID]="+str(sim_id)
delay_df = pd.read_sql(delay_sql, cnxn)
delay_df.columns = ['RunID','Iteration','time','tt_id', 'vehNo', 'travel_time', 'travel_delay']
delay_df = delay_df.loc[delay_df.time>=simulation_start]

In [112]:
delayP_df = delay_df.pivot_table(index = ['RunID','tt_id'], values = 'travel_delay', aggfunc = np.sum).reset_index()
delayP_df = delayP_df.merge(tt_definitions, left_on='tt_id', right_on='Travel_time_id')

delayP_df.loc[:, "delay_coordinated"] = delayP_df.travel_delay * delayP_df.coordinated_through
delayP_df.loc[:, "delay_eb"] = delayP_df.travel_delay * delayP_df.eb
delayP_df.loc[:, "delay_wb"] = delayP_df.travel_delay * delayP_df.wb
delayP_df.loc[:, "delay_total"] = delayP_df.travel_delay * (1-delayP_df.use_for_cdf)

delay_pivot = delayP_df.pivot_table(index = ["RunID"], values = ["delay_coordinated", "delay_total", "delay_eb", "delay_wb"], aggfunc = np.sum)/3600/number_of_intersection


In [134]:
run_ids = [1,2,3,5,4,6,12, 10, 16, 8, 15, 14]
# delay_pivot.reset_index(inplace = True)
# delay_pivot.loc[:, "Scenario"] = scenario
delay_pivot["Non-coordinated delay"] = delay_pivot.delay_total - delay_pivot.delay_coordinated
delay_pivot = delay_pivot.loc[delay_pivot.RunID.isin(run_ids), ["RunID", "delay_eb", "delay_wb", "delay_coordinated", "Non-coordinated delay", "delay_total"]]
delay_pivot.loc[:, "Scenario"] = ["Zero-offset",  "AOG" , "Modified AOG", "Delay" , "AOG-CV", "Number of Stops" , "Bandwidth", "PrOG","Delay + Stops",  "Multiband",  "Maxband", "Kell Method"]

In [135]:
delay_pivot

,RunID,delay_eb,delay_wb,delay_coordinated,Non-coordinated delay,delay_total,Scenario
0,1,36.537731,31.081984,67.619715,72.068434,139.688148,Zero-offset
1,2,28.691023,31.635724,60.326746,71.283865,131.610612,AOG
2,3,28.078896,32.646525,60.725421,71.125628,131.851049,Modified AOG
3,4,29.848300,28.968522,58.816822,72.305333,131.122156,Delay
4,5,28.648707,32.454769,61.103476,69.980661,131.084137,AOG-CV
5,6,29.103562,29.222031,58.325593,72.370692,130.696285,Number of Stops
6,8,23.554088,36.487187,60.041275,69.598184,129.639458,Bandwidth
7,10,43.118236,23.405656,66.523892,69.946786,136.470678,PrOG
8,12,29.263921,29.555999,58.819920,72.455239,131.275159,Delay + Stops
9,14,27.750820,36.813405,64.564225,69.553832,134.118057,Multiband


In [124]:
tt_eb_sql_508 = "SELECT [RunID],[Iteration],[Time],[No_],[Veh],[Trav_] FROM [sim_Andalib].[dbo].[traveltimes] where [StudyID]="+str(sim_id) + "and [No_] = 10508"
tt_eb_508 = pd.read_sql(tt_eb_sql_508, cnxn)
tt_eb_508.columns = ['RunID','Iteration','time','tt_id', 'vehNo', 'travel_time']
tt_eb_508 = tt_eb_508.loc[tt_eb_508.time>=simulation_start]

In [125]:
tt_eb_sql_1208 = "SELECT [RunID],[Iteration],[Time],[No_],[Veh],[Trav_] FROM [sim_Andalib].[dbo].[traveltimes] where [StudyID]="+str(sim_id) + "and [No_] = 11208"
tt_eb_1208 = pd.read_sql(tt_eb_sql_1208, cnxn)
tt_eb_1208.columns = ['RunID','Iteration','time','tt_id', 'vehNo', 'travel_time']
tt_eb_1208 = tt_eb_1208.loc[tt_eb_1208.time>=simulation_start]

In [126]:
tt_wb_sql_511 = "SELECT [RunID],[Iteration],[Time],[No_],[Veh],[Trav_] FROM [sim_Andalib].[dbo].[traveltimes] where [StudyID]="+str(sim_id) + "and [No_] = 10511 "
tt_wb_511 = pd.read_sql(tt_wb_sql_511, cnxn)
tt_wb_511.columns = ['RunID','Iteration','time','tt_id', 'vehNo', 'travel_time']
tt_wb_511 = tt_wb_511.loc[tt_wb_511.time>=simulation_start]

In [127]:
tt_wb_sql_1211 = "SELECT [RunID],[Iteration],[Time],[No_],[Veh],[Trav_] FROM [sim_Andalib].[dbo].[traveltimes] where [StudyID]="+str(sim_id) + "and [No_] = 11211 "
tt_wb_1211 = pd.read_sql(tt_wb_sql_1211, cnxn)
tt_wb_1211.columns = ['RunID','Iteration','time','tt_id', 'vehNo', 'travel_time']
tt_wb_1211 = tt_wb_1211.loc[tt_wb_1211.time>=simulation_start]

In [128]:
tt_eb = pd.merge(left = tt_eb_508, right = tt_eb_1208, how = 'inner', left_on = ["RunID", "Iteration", "vehNo"], right_on = ["RunID", "Iteration", "vehNo"] )
tt_wb = pd.merge(left = tt_wb_1211, right = tt_wb_511, how = 'inner', left_on = ["RunID", "Iteration", "vehNo"], right_on = ["RunID", "Iteration", "vehNo"] )

In [129]:
tt_eb.loc[:, "total_travel_time"] = tt_eb["time_y"] - (tt_eb["time_x"] -tt_eb["travel_time_x"])
tt_wb.loc[:, "total_travel_time"] = tt_wb["time_y"] - (tt_wb["time_x"] -tt_wb["travel_time_x"])

In [157]:
scenario_plot = ["Zero-offset", "AOG" , "Modified-AOG", "AOG-Upstream origin", "Delay" , "Number of Stops" , "Delay + Stops", "PrOG", "Kell Method", "BW" , "MAXBAND", "MULTIBAND" ]
# run_ids = [1, 2,4,6,12, 10, 16, 8, 15, 14]

In [131]:
tt_eb.head()

,RunID,Iteration,time_x,tt_id_x,vehNo,travel_time_x,time_y,tt_id_y,travel_time_y,total_travel_time
0,8,7,474.300182,10508,1779,92.826341,618.841094,11208,26.614242,237.367253
1,8,7,474.431165,10508,1555,100.060391,620.852889,11208,28.979420,246.482115
2,8,7,476.092306,10508,1875,75.614585,617.289825,11208,29.241535,216.812104
3,8,7,476.237822,10508,1880,79.217093,622.017424,11208,29.141719,224.996695
4,8,7,478.095858,10508,1962,58.544921,622.117313,11208,26.601516,202.566377


In [152]:
travel_time_reliability = pd.DataFrame(columns = ["RunID", "median_eb", "q75_eb", "q25_eb", "median_wb", "q75_wb", "q25_wb"])

for i in range(len(run_ids)): 
        
        rid = run_ids[i]
        
        tt_eb_selected = tt_eb.loc[(tt_eb["RunID"]==rid), "total_travel_time"]
        tt_wb_selected = tt_wb.loc[(tt_wb["RunID"]==rid), "total_travel_time"]

        eb_75, eb_50,  eb_25 = np.percentile(tt_eb_selected, [75 ,50, 25])
        wb_75, wb_50,  wb_25 = np.percentile(tt_wb_selected, [75 ,50, 25])
        
        travel_time_reliability.loc[i, :] = [rid, eb_50, eb_75, eb_25, wb_50, wb_75, wb_25]

travel_time_reliability.loc[:, "Scenario"] = scenario_plot

In [153]:
travel_time_reliability

,RunID,median_eb,q75_eb,q25_eb,median_wb,q75_wb,q25_wb,Scenario
0,1,248.483,260.456,235.204,215.034,235.124,192.497,Zero-offset
1,2,215.57,239.967,205.93,219.268,240.147,199.501,AOG
2,3,215.306,229.173,205.58,223.485,243.449,202.423,Modified-AOG
3,5,216.66,244.186,205.917,222.638,241.467,202.12,AOG-Upstream origin
4,4,215.59,257.437,204.156,208.737,233.261,184.272,Delay
5,6,214.422,253.278,203.57,210.38,234.688,185.482,Number of Stops
6,12,214.552,250.723,204.139,210.356,235.779,185.273,Delay + Stops
7,10,263.341,283.25,245.458,162.47,201.032,130.387,PrOG
8,16,257.953,275.628,238.654,204.074,234.066,158.245,Kell Method
9,8,181.379,238.628,162.118,218.905,243.107,204.142,BW


In [158]:
fig, axes = plt.subplots(1, 1, figsize = (5,5))

mark_color = [ "purple","crimson",  "red", "lightsalmon", "goldenrod", "darkorange", "khaki", "darkolivegreen", "lime","green", "black", "grey"]

loosely_dashed = (0, (5,5))

eb = travel_time_reliability.loc[:, "median_eb"]
wb = travel_time_reliability.loc[:, "median_wb"]

for i in range(len(run_ids)):
        
        eb_error = [[eb[i] - travel_time_reliability.loc[i, "q25_eb"]], [travel_time_reliability.loc[i, "q75_eb"]-eb[i]]]
        wb_error = [[wb[i] - travel_time_reliability.loc[i, "q25_wb"]], [travel_time_reliability.loc[i, "q75_wb"]-wb[i]]]
        
        errorbar_line = axes.errorbar(eb[i],wb[i], xerr= eb_error, yerr= wb_error, fmt = 'o', capsize = 2, color = mark_color[i], label =  scenario_plot[i])
        errorbar_line[-1][0].set_linestyle(loosely_dashed)
        errorbar_line[-1][1].set_linestyle(loosely_dashed)
        
        errorbar_line[-1][0].set_linewidth(0.5)
        errorbar_line[-1][1].set_linewidth(0.5)
        
axes.set_ylabel("West bound travel time (in second)", fontsize = 10)
axes.set_xlabel("East bound travel time (in second)", fontsize = 10)

axes.set_xlim([160, 300])
axes.set_ylim([120, 260])

axes.grid(which = 'both', linestyle = 'dashed', alpha = 0.2)
axes.grid(b=True, which='minor', alpha=0.05)
axes.minorticks_on()

fig.legend(scenario_plot, ncol = 4, loc = 'lower center', bbox_transform=fig.transFigure, bbox_to_anchor = (0.52,-0.1))
fig.savefig(directory + "tt_scatterplots.png" , dpi = 300, bbox_inches = "tight")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
 [[eb[0] - travel_time_reliability.loc[0, "q25_eb"]], [travel_time_reliability.loc[0, "q75_eb"]-eb[0]]]

[[13.279624371028234], [11.972284874156344]]

In [141]:
fig, axes = plt.subplots(nrows = 1, ncols = 2,sharey = True, figsize=(8,4))
fig.tight_layout()

seq = np.arange(0,601,5)

title_ = ["Eastbound", "Westbound"]

line = 0

for i in run_ids:
#     run_num = delay_pivot.index[i]
#     if(line%2 or line == 0): 
#                 print(line)
#                 lstyle = 'solid'
#     else: lstyle = 'dashed'
    lstyle = 'solid'
    inbound = tt_eb.loc[(tt_eb.RunID == i) , 'total_travel_time']
    outbound = tt_wb.loc[(tt_wb.RunID == i) , 'total_travel_time']

    inbound = np.histogram(inbound, seq)
    outbound = np.histogram(outbound, seq)

    inbound = np.cumsum(inbound[0])/sum(inbound[0])
    outbound = np.cumsum(outbound[0])/sum(outbound[0])
    
    axes[0].plot(seq[1:], inbound, color = mark_color[line], linestyle = lstyle)
    axes[1].plot(seq[1:], outbound, color = mark_color[line], linestyle = lstyle)
    
    line += 1
    
for i in range(2):
    axes[i].set_xlim([100,350])
    axes[i].grid(which = 'both', linestyle = 'dashed')
    axes[i].grid(b=True, which='minor', alpha=0.2)
    axes[i].minorticks_on()
    axes[i].set_xlabel("Travel Time (in sec)")
    axes[i].set_title(title_[i])
    
axes[0].set_ylabel("Cumulative Probability Distribution")

fig.legend(scenario_plot, ncol = 5, loc = 'lower center', bbox_transform=fig.transFigure, bbox_to_anchor = (0.52,-0.2))

fig.savefig(directory+"cdf"+".png", dpi = 500,  bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [142]:
fig, axes = plt.subplots(2, 1, sharex = True, figsize = (6.5, 6))

eastbound_data = []
westbound_data = []

for i in run_ids:
        
        inbound = tt_eb.loc[(tt_eb.RunID == i) , 'total_travel_time']
        outbound = tt_wb.loc[(tt_wb.RunID == i) , 'total_travel_time']
        
        eastbound_data.append(inbound)
        westbound_data.append(outbound)


axes[0].set_title("Eastbound")
axes[1].set_title("Westbound")
        
mean_dict = {'markerfacecolor':'red', 'markeredgecolor':'r', 'marker':'s', 'markersize':3}
median_dict = {'color':'black', 'linestyle':'--'}   
flier_dict = {'marker': 'o', 'markersize': 1}

axes[0].boxplot(eastbound_data, labels = scenario_plot, showmeans = 1, flierprops=flier_dict, medianprops = median_dict, meanprops =mean_dict )
axes[1].boxplot(eastbound_data, labels = scenario_plot, showmeans = 1, flierprops=flier_dict, medianprops = median_dict, meanprops =mean_dict )
axes[1].tick_params(labelrotation=90)

fig.savefig(directory+"boxplot"+".png", dpi = 500,  bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [147]:
final_results = pd.read_excel("C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\final figure and tables\\final results.xlsx", sheet_name = "final results")

In [148]:
final_results

,Scenario,aog_eb,aog_wb,AOG Total,Stop_eb,Stop_wb,Stop Total,bw_eb,bw_wb,BW Total,prog_eb,prog_wb,PrOG Total,Delay Total
0,Zero-offset,7345.00,6670.24,14015.24,3052.98,2136.71,5189.69,0.0,0.0,0.0,1.04,0.18,1.22,67.62
1,AOG,7775.51,6569.68,14345.19,2413.32,2596.20,5009.52,16.9,0.0,16.9,13.17,1.64,14.81,60.32
2,Modified-AOG,7961.05,6373.00,14334.05,2304.66,2581.45,4886.11,20.9,0.0,20.9,16.21,0.83,17.04,60.72
3,AOG-CV,7877.28,6346.50,14223.78,2447.82,2643.08,5090.90,18.0,0.0,18.0,13.94,0.71,14.65,61.10
4,Delay,7704.51,6561.36,14265.87,2560.59,2368.17,4928.76,8.3,2.5,10.8,6.54,6.87,13.41,58.81
5,Number of Stops,7863.71,6374.84,14238.55,2415.61,2321.19,4736.80,9.3,2.6,11.9,7.13,6.74,13.87,58.33
6,Delay + Stops,7760.70,6556.99,14317.69,2484.96,2339.35,4824.31,10.3,0.8,11.1,8.05,5.95,14.00,58.81
7,PrOG,6979.57,7452.69,14432.26,3842.50,1674.86,5517.36,2.0,51.9,53.9,3.88,43.55,47.43,66.52
8,Kell Method,7250.11,6954.30,14204.41,3506.86,2153.30,5660.16,12.8,15.1,27.9,10.98,12.83,23.81,66.42
9,BW,8394.82,6416.29,14811.11,1823.67,3144.27,4967.94,39.9,19.9,59.8,28.62,13.78,42.40,60.04


In [149]:
fig, axes = plt.subplots(3, 2, figsize = (6.5, 6.5))
fig.tight_layout()
fig.subplots_adjust(hspace=0.4, wspace = 0.2)

# mark_color = [ "blue", "red", "yellow", "darkorange", "peru", "darkolivegreen", "lime","green", "black", "grey"]

for i in range(len(run_ids)):
        axes[0][0].scatter(final_results.loc[i, "AOG Total"], final_results.loc[i, "Stop Total"], color = mark_color[i], label =  scenario_plot[i])
        axes[0][1].scatter(final_results.loc[i, "AOG Total"], final_results.loc[i, "Delay Total"], color = mark_color[i], label =  scenario_plot[i])
        axes[1][0].scatter(final_results.loc[i, "AOG Total"], final_results.loc[i, "PrOG Total"], color = mark_color[i], label =  scenario_plot[i])
        axes[1][1].scatter(final_results.loc[i, "PrOG Total"], final_results.loc[i, "BW Total"], color = mark_color[i], label =  scenario_plot[i])
        axes[2][0].scatter(final_results.loc[i, "BW Total"], final_results.loc[i, "Stop Total"], color = mark_color[i], label =  scenario_plot[i])
        axes[2][1].scatter(final_results.loc[i, "Stop Total"], final_results.loc[i, "Delay Total"], color = mark_color[i], label =  scenario_plot[i])


axes[0][0].set_xlabel("AOG (in number)")
axes[0][0].set_ylabel("Stop (in number)")

axes[0][1].set_xlabel("AOG (in number)")
axes[0][1].set_ylabel("Delay (in veh-hr)")

axes[1][0].set_xlabel("AOG (in number)")
axes[1][0].set_ylabel("PrOG")

axes[1][1].set_xlabel("PrOG")
axes[1][1].set_ylabel("BW")

axes[2][0].set_xlabel("BW")
axes[2][0].set_ylabel("Stop (in number)")

axes[2][1].set_xlabel("Stop (in number)")
axes[2][1].set_ylabel("Delay (in veh-hr)")

axes[0][0].set_title("AOG vs. Stop", fontsize = 10)
axes[0][1].set_title("AOG vs. Delay", fontsize = 10)
axes[1][0].set_title("AOG vs. PrOG", fontsize = 10)
axes[1][1].set_title("PrOG vs. BW", fontsize = 10)
axes[2][0].set_title("BW vs. Stop", fontsize = 10)
axes[2][1].set_title("Stop vs. Delay", fontsize = 10)

fig.legend(scenario_plot, ncol = 5, loc = 'lower center', bbox_transform=fig.transFigure, bbox_to_anchor = (0.52,-0.1))
fig.savefig(directory + "scatterplots.png" , dpi = 300, bbox_inches = "tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
handles, labels = axes[0][0].get_legend_handles_labels()

In [26]:
labels

['Zero-offset',
 'AOG',
 'Delay',
 'Number of Stops',
 'Delay + Stops',
 'PrOG',
 'Kell Method',
 'BW',
 'Maxband',
 'Multiband']

In [27]:
axes[0][0].legend()

In [28]:
final_results.loc[0, "AOG Total"]

14015.24

In [29]:
scenario_plot

['Zero-offset',
 'AOG',
 'Delay',
 'Number of Stops',
 'Delay + Stops',
 'PrOG',
 'Kell Method',
 'BW',
 'Maxband',
 'Multiband']